Run this cell to set everything up:

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import holoviews as hv
import panel as pn
import panel.widgets as pnw
hv.extension("bokeh")
import os

# Function to read files
def read_fukushima_data(file_path):
    return pd.read_csv(file_path, 
                 skiprows=[0,1,2],
                 index_col=0, 
                 parse_dates=True,
                 na_values=["ND","-"],
                 encoding="utf-8")

# read all files in a directory (or in this case based on wildcards)
file_list = glob("datafiles/*Unit[123]*")
#df_list = [read_fukushima_data(f) for f in file_list]

# read the files into a dictionary of dataframes (they keys will be used in the GUI panels)
df_dict = {}
for f in file_list:
    key = os.path.splitext(os.path.basename(f))[0]
    df_dict[key] = read_fukushima_data(f)
    
file_labels = sorted(df_dict.keys())

# only columns that each file has will be plottable
common_columns = df_dict[file_labels[0]].columns
for k in file_labels[1:]:
    common_columns = common_columns.intersection(df_dict[k].columns)
    
columns = sorted(common_columns)
discrete = [x for x in columns if df_dict[file_labels[0]][x].dtype == object]
continuous = [x for x in columns if x not in discrete]
quantileable = [x for x in continuous if len(df_dict[file_labels[0]][x].unique()) > 20]

# method to create a panel
x = "Date"
def create_comparison():
    y = pnw.CheckBoxGroup(name="Y-Axis", value=[quantileable[0]], options=quantileable, width_policy="fit")
    file_label = pnw.CheckBoxGroup(name="File", value=[file_labels[0]], options=file_labels, width_policy="fit")

    @pn.depends(file_label.param.value, y.param.value)
    def create_figure(file_label, y):
        if (len(file_label) == 0) or (len(y) == 0):
                return
        curve_list = []
        for fl in file_label:
            curve_list += [hv.Curve(df_dict[fl], [x, y], label="{}, {}".format(fl, y)) for y in y]
        return hv.Overlay(curve_list).opts(frame_width=600, frame_height=450, legend_position="right")

    #widgets = pn.WidgetBox(year, y)

    return pn.Row(pn.Column("### File:", file_label, "### Y-Axis:", y, width_policy="fit"), create_figure, width_policy="fit")

Just run the "create_comparison" method to create a panel.

In [ ]:
create_comparison()

In [ ]:
create_comparison()

In [ ]:
create_comparison()